<h3> Extracting protein sequences' features using Albert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
import torch
from transformers import AlbertModel, AlbertTokenizer
import re

<b>2. Set the file location of Albert and the vocabulary file<b>

In [2]:
modelPath = '/media/agemagician/Disk2/projects/bio_google_tpu_project/ahmed/models_pytorch/albert'
vocabPath = '/media/agemagician/Disk2/projects/bio_google_tpu_project/ahmed/models_pytorch/albert/albert_vocab_model.model'

<b>3. Load the vocabulary and Albert Model<b>

In [3]:
vocab = AlbertTokenizer(vocabPath, do_lower_case=False )

In [4]:
model = AlbertModel.from_pretrained(modelPath)

<b>4. Load the model into the GPU if avilabile and switch to inference mode<b>

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
model = model.to(device)
model = model.eval()

<b>5. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [7]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [8]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>6. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [9]:
ids = vocab.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

In [10]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

<b>7. Extracting sequences' features and load it into the CPU if needed<b>

In [11]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

In [12]:
embedding = embedding.cpu().numpy()

<b>8. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by Albert model<b>

In [13]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [14]:
print(features)

[array([[-0.06913061,  0.03206101, -0.05003147, ..., -0.11457862,
        -0.03595566,  0.08152848],
       [-0.33643803, -0.0175079 ,  0.10670088, ..., -0.20067249,
         0.17586312,  0.15829395],
       [-0.17356758,  0.03067275, -0.09666996, ..., -0.1891188 ,
         0.10043841,  0.14909169],
       ...,
       [-0.11075828, -0.19667947, -0.02256219, ..., -0.10256966,
         0.08064437, -0.2150105 ],
       [-0.11349688, -0.02344881, -0.02433204, ..., -0.19947116,
        -0.01309577,  0.06546066],
       [-0.08537761, -0.19285612, -0.01876267, ..., -0.08882213,
         0.07482602, -0.19948626]], dtype=float32), array([[-0.12822625,  0.10716017, -0.01129915, ..., -0.11677967,
         0.03904792,  0.10010004],
       [-0.36793104,  0.03902491, -0.1156951 , ..., -0.2210128 ,
         0.10926703,  0.13324448],
       [-0.26249766,  0.11969337, -0.05417463, ..., -0.20998038,
         0.11928435,  0.21757801],
       [-0.03641748, -0.08373182,  0.05713482, ..., -0.12186566,
     